<a href="https://colab.research.google.com/github/rajgupt/my-colab-notebooks/blob/master/orientation_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
import os
import numpy as np
import IPython.display as display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from PIL import Image

In [0]:
# Ref: https://d4nst.github.io/2017/01/12/image-orientation/

In [0]:
os.chdir('/content/drive/My Drive/colab/data')

In [0]:
!ls

Data source: https://guillaumejaume.github.io/FUNSD/dataset.zip

# Dataset Preparation

In [0]:
!mkdir -p funsd_dataset/rotate/train/0

In [0]:
!cp funsd_dataset/training_data/images/* funsd_dataset/rotate/train/0/

In [0]:
from PIL import Image
import glob
# if not os.path.exists('funsd_dataset/rotate'):
images = glob.glob('funsd_dataset/rotate/train/0/*.png')
os.makedirs('funsd_dataset/rotate/train/90', exist_ok=True)
os.makedirs('funsd_dataset/rotate/train/180', exist_ok=True)
os.makedirs('funsd_dataset/rotate/train/270', exist_ok=True)
for angle in [90,180,270]:
  for im in images:
    img = Image.open(im)
    imname = os.path.basename(im)
    save_path = os.path.join('funsd_dataset/rotate/train/'+str(angle)+'/'+imname)
    img.rotate(angle, expand=True).save(save_path)

# Orientation Prediction Model

Basic Classifier to predict the angle of orientation

In [0]:
import pathlib
data_dir = pathlib.Path('funsd_dataset/rotate/train')

In [0]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*.png'))
list_ds

In [0]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*')])
CLASS_NAMES

In [0]:
correct = list(data_dir.glob('0/*'))
display.display(Image.open(str(correct[0])))

In [0]:
train_datagen = ImageDataGenerator(
    rotation_range=10,
    channel_shift_range=20,
    horizontal_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

In [0]:
BATCH_SIZE = 32
IMG_SIZE = 224
# STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [0]:
train_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size  = BATCH_SIZE,
    color_mode='grayscale',
    class_mode  = 'categorical',
    interpolation = 'lanczos', # <--------- random crop
    shuffle = True,
    seed=1337,
    subset="training"
)

val_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size  = BATCH_SIZE,
    color_mode='grayscale',
    class_mode  = 'categorical',
    interpolation = 'lanczos', # <--------- random crop
    shuffle = True,
    seed=1337,
    subset="validation"
)

In [0]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n][:,:,0], cmap='gray')
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [0]:
image_batch, label_batch = next(train_ds)
show_batch(image_batch, label_batch)

In [0]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, CenterCrop

In [0]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    # x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [0]:
train_ds[0][0].shape, train_ds[0][1].shape

In [0]:
train_tfds = tf.data.Dataset.from_generator(train_datagen.flow_from_directory,
                                            (tf.int64, tf.int64), 
                                            (tf.TensorShape([32,224,224,1]), tf.TensorShape([32,4])), args=[str(datadir)]) 
train_tfds

In [0]:
list(train_tfds.take(1).as_numpy_iterator()) 

In [0]:
# with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
#   model = make_model((IMG_SIZE,IMG_SIZE,1), 4)
#   epochs = 1
#   model.summary()

#   callbacks = [
#       keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
#   ]
#   model.compile(
#       optimizer=keras.optimizers.Adam(1e-3),
#       loss="binary_crossentropy",
#       metrics=["accuracy"],
#   )
#   model.fit(
#       train_tfds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
#   )

In [0]:
with tf.device('/device:GPU:0'):
  model = make_model((IMG_SIZE,IMG_SIZE,1), 4)
  epochs = 1
  model.summary()
  callbacks = [
      keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
  ]
  model.compile(
      optimizer=keras.optimizers.Adam(1e-3),
      loss="binary_crossentropy",
      metrics=["accuracy"],
  )
  model.fit(
      train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
  )